In [1]:
import time
import pathlib
import pandas as pd 
from selenium import webdriver

Make DataFrame

In [2]:
df_youtube = pd.DataFrame(columns = ['date_published', 'title', 'views', 'keywords', 'video_id'])

Define function for scraping the following data from Youtube: date, title, views and ID

In [12]:
def youtube_channel(extension, count, df=df_youtube): 
    '''
    Transforming df_youtube DataFrame:
    1) Initialize and run chromedriver;
    2) Find the chanel, open videos tab and scrape date, title, views and ID of "count"-number latest videos;
    3) Assign each data category to each column in DataFrame.
    '''
    
    driver = webdriver.Chrome('chromedriver\\chromedriver.exe')
    youtube = 'https://www.youtube.com/'
    driver.get(f'{youtube}results?search_query={extension}')
    time.sleep(1.5)
    channel_entrance = driver.find_element_by_css_selector('#main-link')
    channel_entrance.click()
    time.sleep(1.5)
    channel_videos_entrance = driver.find_element_by_css_selector('#tabsContent > tp-yt-paper-tab:nth-child(4) > div')
    channel_videos_entrance.click()
    time.sleep(2.5)
    part_one = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['
    part_two = ']/div[1]/div[1]/div[1]/h3/a'
    date_list, title_list, views_list, id_list  = [], [] , [] ,[]
    
    for r in range(1, count + 1):
        url = part_one + str(r) + part_two
        videos_entrance = driver.find_element_by_xpath(url)
        driver.execute_script("arguments[0].click();", videos_entrance)
        time.sleep(1.5)
        date_list.append(driver.find_element_by_css_selector('#date > yt-formatted-string').text)
        title_list.append(driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text)
        views_list.append(driver.find_element_by_css_selector('#count > ytd-video-view-count-renderer > span.view-count.style-scope.ytd-video-view-count-renderer').text)
        time.sleep(2.5)
        video_ids = driver.find_elements_by_xpath('//*[@id="page-manager"]/ytd-watch-flexy')
        
        for video in video_ids:
            video_id = str(video.get_attribute('video-id'))
            id_list.append(video_id)
        driver.back()
        
    df['date_published'] = date_list
    df['title'] = title_list
    df['views'] = views_list
    df['video_id'] = id_list
    driver.quit()

Define function to scrape keywords

In [14]:
def youtube_keywords(df=df_youtube, overall_list = [] ):
    '''
    Transforming df_youtube DataFrame:
    1) Initialize and run chromedriver;
    2) Scrape tags through each video from DataFrame's video_id column;
    3) Assign each list of tags to coresponding column in DataFrame.
    '''
    
    for video_id in list(df['video_id']):
        tag_list = [] 
        driver = webdriver.Chrome('chromedriver\\chromedriver.exe')
        youtube = 'https://www.youtube.com/'
        driver.get(f'{youtube}watch?v={video_id}')
        time.sleep(2)
        tags = driver.find_elements_by_xpath("//meta[@property='og:video:tag']") 
        
        for tag in tags:
            video_tag = str(tag.get_attribute('content'))     
            tag_list.append(video_tag)  
        overall_list.append(tag_list)   
        driver.quit() 
    
    for i in range(len(df['keywords'])):
        df['keywords'][i] = overall_list[i]

Run **youtube_channel()** function to extract date, title, views and ID

In [13]:
youtube_channel(input('Enter your channel: ').replace(' ','+'), int(input('Enter amount of latest videos: ')))
df_youtube

Enter your channel: skirmantas malinauskas
Enter amount of latest videos: 5


,date_published,title,views,keywords,video_id
0,14 Mar 2021,Henrikas Daktaras: Mes su Šviniumi buvome vers...,"85,208 views",NaN,gQayDUCH05Q
1,7 Mar 2021,Kaip pavogti milijoną?,"76,196 views",NaN,AdUiTzUNrd8
2,28 Feb 2021,Apie tai kalbėti draudžiama,"85,305 views",NaN,Aq-bpR6vFig
3,21 Feb 2021,Safari Kaune ir antstoliai prie Šviniaus durų,"128,298 views",NaN,2l6-mS6pt0c
4,16 Feb 2021,"""CityBee"" mums pamelavo - 110 tūkstančių žmoni...","147,787 views",NaN,wbKznbxe5r4


Run **youtube_keywords()** function to extract tags directly to *df_youtube* DataFrame

In [15]:
youtube_keywords()
df_youtube

,date_published,title,views,keywords,video_id
0,14 Mar 2021,Henrikas Daktaras: Mes su Šviniumi buvome vers...,"85,208 views","[henrikas daktaras, henrikas, pukelis, arūnas ...",gQayDUCH05Q
1,7 Mar 2021,Kaip pavogti milijoną?,"76,196 views","[Geostatyba, Nerijus Vaitonis, Sigita Astrauki...",AdUiTzUNrd8
2,28 Feb 2021,Apie tai kalbėti draudžiama,"85,305 views","[Algirdas Toliatas, Paulius Vaineikis, Valdas ...",Aq-bpR6vFig
3,21 Feb 2021,Safari Kaune ir antstoliai prie Šviniaus durų,"128,298 views","[arūnas pukelis, arunas pukelis, lff, švinius,...",2l6-mS6pt0c
4,16 Feb 2021,"""CityBee"" mums pamelavo - 110 tūkstančių žmoni...","147,787 views","[CityBee, asmens duomenys, vagystė, automobili...",wbKznbxe5r4
